In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold, train_test_split
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import timm
from torchmetrics import Metric
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from einops.layers.torch import Rearrange
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import random

In [38]:
class NMAE(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("nmae", default=torch.tensor(0.), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")
    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape
        nmae = torch.sum(torch.abs(target-preds) / (torch.abs(target)))
        n_obs = target.numel()
        self.nmae += nmae
        self.total += n_obs
    def compute(self):
        return self.nmae / self.total

In [3]:
def Image_Data(size):
    preprocessing_train_imgs = glob(f"D:/DACON_GROWTH/PREPROCESSING-TRAIN_{size}/*")
    preprocessing_test_imgs = glob(f"D:/DACON_GROWTH/PREPROCESSING-TEST_{size}/*")
    augmetation_imgs = glob(f"D:/DACON_GROWTH/AUGMENTATION/SIZE{size}/*")

    train_path = []
    train_imgs = []
    for path in preprocessing_train_imgs:
        img_file = os.path.basename(path)
        img_name = img_file.split(".")[0]
        case = img_name.split("_")[0]
        if case in ["CASE59"]:
            continue
        elif img_name in ["CASE05_21", "CASE10_41", "CASE45_16", "CASE45_17"]:        
            continue
        else:
            img = cv2.imread(path, cv2.IMREAD_COLOR)
            train_path.append(img_name)
            train_imgs.append(img)

    test_path = []
    test_imgs = []
    for path in preprocessing_test_imgs:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        test_path.append(path)
        test_imgs.append(img)

    augment_path = []
    augment_imgs = []
    for path in augmetation_imgs:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        augment_path.append(path)
        augment_imgs.append(img)

    augment_path = augment_path 
    augment_imgs = augment_imgs 
    return train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs

In [4]:
main_path = "D:/DACON_GROWTH"
train_data = glob(main_path+"/train/*/meta/*.csv")
train_label = glob(main_path+"/train/*/*.csv")
test_data = glob(main_path+"/test/meta/*.csv")

def make_data(train_data, golden_dict=None):
    train_df = []
    for i in tqdm(train_data):
        name = i.split("\\")[-1].split(".")[0]
        df = pd.read_csv(i)
        df["이미지"] = name
        if not golden_dict:
            case = name.split("_")[0]
            label = pd.read_csv(f"D:/DACON_GROWTH/train/{case}/label.csv")
            label_name = [i.split(".")[0] for i in label.img_name]
            label.img_name = label_name
            leaf_weight = label[label.img_name == name].leaf_weight.values[0]
            df["무게"] = leaf_weight
            df["무게"] = df["무게"].apply(lambda x: 0.001 if x<0.001 else x)
        df = df.replace([np.inf, -np.inf], np.nan)
        df = df.interpolate()
        df = df.fillna(0.0)
        train_df.append(df)

    train_df = pd.concat(train_df, ignore_index=True)
    train_df.drop(columns=["화이트 LED동작강도", "레드 LED동작강도", 
                      "블루 LED동작강도", "최근분무량"], inplace=True)
    train_df["시간2"] = train_df["시간"].apply(lambda x: x.split(" ")[-1])
    if not golden_dict:
        golden_dict ={}
        for e, i in enumerate(train_df["시간2"].unique()):
               golden_dict[i] = e
    train_df["순서"] = train_df["시간2"].apply(lambda x : golden_dict[x])
    train_df = train_df[(train_df["순서"] > 894) & (train_df["순서"] < 919)]
    train_df.drop(columns=["시간", "시간2"], inplace=True)
    return train_df, golden_dict

train_df, golden_dict = make_data(train_data)
test_df, golden_dict = make_data(test_data, golden_dict)

img2weight = {}
for i in tqdm(train_df["이미지"].unique()):
    temp = train_df[train_df["이미지"]==i]
    weight = temp["무게"].values[0]
    img2weight[i] = weight

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1592/1592 [00:06<00:00, 264.13it/s]


In [11]:
features = ['내부온도관측치', '외부온도관측치', '내부습도관측치', '외부습도관측치', 'CO2관측치', 'EC관측치',
       '냉방온도', '냉방부하',
       '난방온도', '난방부하', '총추정광량', '백색광추정광량', '적색광추정광량', '청색광추정광량']

In [26]:
test_df

,내부온도관측치,외부온도관측치,내부습도관측치,외부습도관측치,CO2관측치,EC관측치,냉방온도,냉방부하,난방온도,난방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량,이미지,순서
522,25.500000,22.400000,76.000000,37.099998,368.0,0.000000,25.000000,2.249999,23.0,0.0,363.9270,281.5631,46.3344,36.0295,001,895
523,25.799999,22.400000,76.699997,37.000000,369.0,0.000000,25.000000,2.500000,23.0,0.0,363.9270,281.5631,46.3344,36.0295,001,896
524,26.299999,22.799999,77.800003,37.000000,359.0,0.000000,25.000000,5.500002,23.0,0.0,363.9270,281.5631,46.3344,36.0295,001,897
525,26.299999,22.900000,78.099998,36.900002,356.0,0.000000,25.000000,6.499996,23.0,0.0,363.9270,281.5631,46.3344,36.0295,001,898
526,26.000000,23.000000,77.400002,36.799999,358.0,0.000000,25.000000,6.499996,23.0,0.0,363.9270,281.5631,46.3344,36.0295,001,899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661874,27.799999,27.600000,84.099998,52.599998,350.0,0.214662,17.983524,68.999996,14.0,0.0,357.7882,278.4690,47.9892,31.3300,460,914
661875,27.799999,27.600000,84.099998,52.700001,341.0,0.224453,17.949529,68.999996,14.0,0.0,353.1276,275.3749,47.9892,29.7635,460,915
661876,27.900000,27.600000,81.800003,52.500000,344.0,0.229725,17.915917,69.499998,14.0,0.0,353.1276,275.3749,47.9892,29.7635,460,916
661877,27.900000,27.600000,75.599998,52.500000,396.0,0.255334,17.881675,69.499998,14.0,0.0,353.1276,275.3749,47.9892,29.7635,460,917


In [27]:
img2data = {}
for i in tqdm(train_df["이미지"].unique()):
       temp = train_df[train_df["이미지"]==i]
       temp_mean = temp[features].mean(axis=0)
       img2data[temp["이미지"].values[0]] = np.array(temp_mean[["EC관측치",'내부습도관측치', '외부습도관측치']].values, dtype=np.float16)
img2test = {}
for i in tqdm(test_df["이미지"].unique()):
       temp = test_df[test_df["이미지"]==i]
       temp_mean = temp[features].mean(axis=0)
       img2test[temp["이미지"].values[0]] = np.array(temp_mean[["EC관측치",'내부습도관측치', '외부습도관측치']].values, dtype=np.float16)


 79%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                       | 1260/1592 [00:07<00:01, 172.34it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1592/1592 [00:09<00:00, 171.40it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 460/460 [00:01<00:00, 306.87it/s]


In [32]:
for k, v in img2data.items():
    print(v.shape)

(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)


In [33]:
class PYTORCH_DATASET(nn.Module):
    def __init__(self, path: list, img2weight: dict, imgs: np.array, img2data: dict,  \
                 transform=None):
        super(PYTORCH_DATASET, self).__init__()
        self.path = path
        self.imgs = imgs

        self.transform = transform
        self.img2weight = img2weight
        self.img2data = img2data

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        before_img = self.imgs[idx]
        img_path = self.path[idx]
        img_path = img_path.split("\\")[-1].split(".")[0]
        df_index = self.img2data[img_path]
        df_label = self.img2weight[img_path]

        after_img = self.transform(image=before_img)["image"]

        return {"사진": after_img, "정보": df_index, "상표": df_label}

class PYTORCH_PREDICTSET(nn.Module):
    def __init__(self, path: list,  imgs: np.array, img2data: dict, \
                 transform=None):
        super(PYTORCH_PREDICTSET, self).__init__()
        self.path = path
        self.imgs = imgs

        self.transform = transform
        self.img2data = img2data

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        before_img = self.imgs[idx]
        img_path = self.path[idx]
        img_path = img_path.split("\\")[-1].split(".")[0]
        df_index = self.img2data[img_path]
        after_img = self.transform(image=before_img)["image"]
        return {"사진": after_img, "정보": df_index}

class PYTORCH_AUGMENTSET(nn.Module):
    def __init__(self, aug_imgs: np.array, transform_ori = None, transform_aug=None):
        super(PYTORCH_AUGMENTSET, self).__init__()
        self.aug_imgs = aug_imgs
        self.transform_ori = transform_ori
        self.transform_aug = transform_aug

    def __len__(self):
        return len(self.aug_imgs)

    def __getitem__(self, idx):
        before_img = self.aug_imgs[idx]
        after_img = self.transform_aug(image=before_img)["image"]
        before_img = self.transform_ori(image=before_img)["image"]
        temp_data = torch.FloatTensor([[0, 0, 0]])

        return {"원본": before_img, "확장": after_img, "정보": temp_data}

In [14]:
def Augmentation_Policy(train_size, test_size):
    test_image_size = train_size
    train_image_size = test_size

    train_mode= albu.Compose([
        albu.Resize(train_image_size, train_image_size),
        albu.RandomResizedCrop(height=train_image_size, width=train_image_size,
                               scale=(0.25, 1.0), ratio=(0.75, 1.3333333333333333),
                               interpolation=1, p=1),
        albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
                              rotate_limit=30, interpolation=1, border_mode=0, value=0, p=0.25),
        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),
        albu.OneOf([
            albu.MotionBlur(p=.2),
            albu.MedianBlur(blur_limit=3, p=0.1),
            albu.Blur(blur_limit=3, p=0.1),
#             albu.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.4,p=0.2)
        ], p=0.25),
        albu.OneOf([
            albu.CLAHE(clip_limit=2),
            albu.Sharpen(),
            albu.Emboss(),
            albu.RandomBrightnessContrast(),
        ], p=0.25),
        albu.CoarseDropout(max_holes=8, max_height=8, max_width=8, fill_value=0, p=0.25),
        albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
        ToTensorV2(),
    ])

    test_mode = albu.Compose([
        albu.Resize(test_image_size, test_image_size),
        albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
        ToTensorV2(),
    ])
    return train_mode, test_mode

In [49]:
class Regressor(nn.Module):
    def __init__(self, model_type, out_features):
        super(Regressor, self).__init__()
        self.model_type = model_type
        if model_type in ["efficientnet-b5", "efficientnet-b6"]:
            self.model = EfficientNet.from_pretrained(model_type, num_classes=1)
        else:
            self.model =  timm.create_model(model_type, pretrained=True, num_classes=1)
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(out_features+3, 128)
        self.output = nn.Linear(128, 1)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(0.5)
        self.dropout1 = nn.Dropout(0.65)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0)

    def forward(self,  x, x2):
        if self.model_type in ["efficientnet-b5", "efficientnet-b6"]:
            x = self.model.extract_features(x)
        else:
            x = self.model.forward_features(x)
        x = self.avg(x)
        x = self.dropout(x)
        x = x.flatten(start_dim=1, end_dim=-1)
        x2 = x2.view(-1, 3)
        if self.training:
            c = random.choice([0, 0, 0, 1, 2, 3])
            if c == 0:
                pass
            elif c == 1:
                x2 = self.dropout1(x2)
            elif c == 2:
                x2 = self.dropout2(x2)
            elif c == 3:
                x2 = self.dropout3(x2)
        x = torch.cat([x, x2], dim=-1)
        x = self.linear(x)
        x = self.gelu(x)
        x = self.output(x)
        return x

In [50]:
from pytorch_lightning.trainer.supporters import CombinedLoader

class LIGHTNING_MODEL(pl.LightningModule):
    def __init__(self, model, model_type, out_features, train_dataset, val_dataset, augment_dataset, 
        predict_dataset, train_batch, valid_batch, augment_batch, predict_batch, learning_rate):
        super(LIGHTNING_MODEL, self).__init__()
        self.model = model(model_type, out_features)
        self.train_nmae = NMAE()
        self.valid_nmae = NMAE()
        self.augment_dataset= augment_dataset
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.predict_dataset = predict_dataset
        self.train_batch = train_batch
        self.valid_batch = valid_batch
        self.augment_batch = augment_batch
        self.predict_batch = predict_batch
        self.learning_rate = learning_rate
        self.bestnmae = 10.
        
    def forward(self, x):
        output = self.model(x)
        return output
    
    def train_dataloader(self):
        loaders = {"a": DataLoader(self.train_dataset, shuffle=True, batch_size=self.train_batch, num_workers=0), \
                   "b": DataLoader(self.augment_dataset, shuffle=True, batch_size=self.augment_batch, num_workers=0)}
        combined_loaders = CombinedLoader(loaders, mode="max_size_cycle")
        return combined_loaders
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, shuffle=False, batch_size=self.valid_batch, num_workers=0)
    
    def predict_dataloader(self):
        return DataLoader(self.predict_dataset, shuffle=False, batch_size=self.predict_batch, num_workers=0)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        ori_batch = train_batch["a"]
        aug_batch = train_batch["b"]
        img = ori_batch["사진"]
        data = ori_batch["정보"]
        label = ori_batch["상표"]
        
        ori = aug_batch["원본"]
        ext = aug_batch["확장"]
        temp_data = aug_batch["정보"]

        pred = self.model(img, data)
        pred = pred.view(-1)
        label = label.view(-1)
        
        train_loss = nn.L1Loss()(pred, label)
        self.log("train_loss", train_loss)
        
        train_nmae = self.train_nmae(pred, label)
        self.log('train_NMAE', train_nmae)
        
        with torch.no_grad():
            ori_out = self.model(ori, temp_data)
            ext_out = self.model(ext, temp_data)
        con_loss = nn.L1Loss()(ori_out, ext_out)
        con_loss = 0.25 * con_loss 
        self.log('con_loss', con_loss)
        loss = train_loss + con_loss
        self.log('loss', loss)
        return loss

    def training_epoch_end(self, outputs):
        train_nmae = self.train_nmae.compute()
        self.log("epoch_train_nmae", train_nmae)
        self.train_nmae.reset()
        print(f"training nmae: {train_nmae:.4}, epoch: {self.trainer.current_epoch}")
                
    def validation_step(self, val_batch, batch_idx):
        img = val_batch["사진"]
        data = val_batch["정보"]
        label = val_batch["상표"]

        pred = self.model(img, data)
        pred = pred.view(-1)
        label = label.view(-1)
        loss = nn.L1Loss()(pred, label)
        self.log("val_loss", loss)
        self.valid_nmae.update(pred, label)
        return loss

    def validation_epoch_end(self, outputs):
        val_nmae = self.valid_nmae.compute()
        self.log("val_nmae", val_nmae)
        ext = val_nmae.item()
        if ext < self.bestnmae:
            self.bestnmae = ext
        self.valid_nmae.reset()
        print(f"validation nmae: {val_nmae:.4}")

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        img = batch["사진"]
        data = batch["정보"]
        pred = self.model(img, data).unsqueeze(0)
        return pred

In [51]:
from efficientnet_pytorch import EfficientNet

In [52]:
model_types = ["efficientnet-b5", "efficientnetv2_rw_s"]
for model_type in model_types:
    if model_type == "efficientnet_b0":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(224)
    if model_type == "efficientnet_b1":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(256)
    if model_type == "efficientnet_b2":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(288)
    if model_type == "efficientnet_b3":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(320)
    if model_type == "efficientnet_b4":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(384)
    if model_type == "efficientnetv2_rw_t":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(288)
    if model_type == "efficientnetv2_rw_s":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(384)
    if model_type == "efficientnetv2_rw_m":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(416)

    if model_type == "efficientnet_b0":
        train_mode, test_mode = Augmentation_Policy(196, 224)
    if model_type == "efficientnet_b1":
        train_mode, test_mode = Augmentation_Policy(224, 256)
    if model_type == "efficientnet_b2":
        train_mode, test_mode = Augmentation_Policy(256, 288)
    if model_type == "efficientnet_b3":
        train_mode, test_mode = Augmentation_Policy(288, 320)
    if model_type == "efficientnet_b4":
        train_mode, test_mode = Augmentation_Policy(320, 384)
    if model_type == "efficientnetv2_rw_t":
        train_mode, test_mode = Augmentation_Policy(224, 288)
    if model_type == "efficientnetv2_rw_s":
        train_mode, test_mode = Augmentation_Policy(288, 384)
    if model_type == "efficientnetv2_rw_m":
        train_mode, test_mode = Augmentation_Policy(320, 416)

    if model_type == "efficientnet_b0":
        out_features = 1280
    if model_type == "efficientnet_b1":
        out_features = 1280
    if model_type == "efficientnet_b2":
        out_features = 1408
    if model_type == "efficientnet_b3":
        out_features = 1536
    if model_type == "efficientnet_b4":
        out_features = 1792
    if model_type == "efficientnetv2_rw_t":
        out_features = 1024
    if model_type == "efficientnetv2_rw_s":
        out_features = 1792
    if model_type == "efficientnetv2_rw_m":
        out_features = 2152
        
    if model_type == "efficientnet-b5":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(456)
    if model_type == "efficientnet-b6":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(528)
    if model_type == "efficientnet-b7":
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(600)

    if model_type == "efficientnet-b5":
        train_mode, test_mode = Augmentation_Policy(384, 456)
    if model_type == "efficientnet-b6":
        train_mode, test_mode = Augmentation_Policy(456, 528)
    if model_type == "efficientnet-b7":
        train_mode, test_mode = Augmentation_Policy(528, 600)

    if model_type == "efficientnet-b5":
        out_features = 2048
    if model_type == "efficientnet-b6":
        out_features = 2304
    if model_type == "efficientnet-b7":
        out_features = 2560
        
    kf = KFold(n_splits=15, shuffle=False)
    for fold, (ti, vi) in enumerate(kf.split(train_path)): 
        if fold == 0:
            trainset_path, validset_path = np.array(train_path)[ti], np.array(train_path)[vi]
            train_imgset, valid_imgset = np.array(train_imgs)[ti], np.array(train_imgs)[vi]
            train_dataset = PYTORCH_DATASET(path=trainset_path, img2weight=img2weight, imgs=train_imgset, img2data=img2data,\
                                                transform=train_mode)
            valid_dataset = PYTORCH_DATASET(path=validset_path, img2weight=img2weight, imgs=valid_imgset, img2data=img2data,\
                                                        transform=test_mode)
            augment_dataset = PYTORCH_AUGMENTSET(aug_imgs=augment_imgs, transform_ori=test_mode, transform_aug=train_mode)
            total_dataset = PYTORCH_DATASET(path=train_path, img2weight=img2weight, imgs=train_imgs, img2data=img2data,\
                                                transform=train_mode)
            test_dataset = PYTORCH_PREDICTSET(path=test_path, imgs=test_imgs, img2data=img2test, \
                                                transform=test_mode)
            model = LIGHTNING_MODEL(model = Regressor, model_type = model_type, out_features = out_features, 
                                    train_dataset = train_dataset, val_dataset = valid_dataset, augment_dataset = augment_dataset, 
                                    predict_dataset = test_dataset, train_batch = 16, valid_batch = 16, augment_batch = 4,
                                    predict_batch = 16, learning_rate = 1e-3)
            check_path =  f"D:/DACON_GROWTH/models/{model_type}"
            finetune_path = f"D:/DACON_GROWTH/models/finetuned/{model_type}"
            if not os.path.exists(check_path):
                os.makedirs(check_path)
                os.makedirs(finetune_path)
            checkpoint_callback = ModelCheckpoint(dirpath=check_path, save_top_k=1, monitor="val_nmae", \
                                                  save_weights_only=True)
            trainer = pl.Trainer(devices=[0], accelerator='gpu', max_epochs=60,
                callbacks=[checkpoint_callback], precision=16)
            trainer.fit(model)
            bestscore = model.bestnmae
            checkpoint_ = os.listdir(check_path)[0]
            file_name = os.path.basename(checkpoint_).split(".ckpt")[0]
            src = os.path.join(check_path, checkpoint_)
            score_name = file_name + f"_{bestscore : .4f}.ckpt"
            dst = os.path.join(check_path, score_name)
            os.rename(src, dst)
            #fine_tuned
#             model = LIGHTNING_MODEL.load_from_checkpoint(checkpoint_path=dst, \
#                     model= Regressor, model_type = model_type, out_features = out_features, \
#                     train_dataset = total_dataset, val_dataset = valid_dataset, \
#                     augment_dataset= augment_dataset, predict_dataset = test_dataset, \
#                     train_batch = 16, valid_batch = 16, augment_batch = 4, predict_batch = 16, learning_rate = 1e-4)
#             checkpoint_callback = ModelCheckpoint(dirpath=finetune_path, save_top_k=3, monitor="val_nmae", save_weights_only=True)
#             trainer = pl.Trainer(devices=[0], accelerator='gpu', max_epochs=6, callbacks=[checkpoint_callback], precision=16)
#             trainer.fit(model)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded pretrained weights for efficientnet-b5



  | Name            | Type               | Params
-------------------------------------------------------
0 | model           | Regressor          | 28.6 M
1 | train_nmae      | NMAE               | 0     
2 | valid_nmae      | NMAE               | 0     
3 | augment_dataset | PYTORCH_AUGMENTSET | 0     
4 | train_dataset   | PYTORCH_DATASET    | 0     
5 | val_dataset     | PYTORCH_DATASET    | 0     
6 | predict_dataset | PYTORCH_PREDICTSET | 0     
-------------------------------------------------------
28.6 M    Trainable params
0         Non-trainable params
28.6 M    Total params
57.211    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

validation nmae: 1.004


...

validation nmae: 0.3308
training nmae: 3.291, epoch: 57


Validation: 0it [00:00, ?it/s]

validation nmae: 0.3253
training nmae: 3.786, epoch: 58


Validation: 0it [00:00, ?it/s]

validation nmae: 0.2187
training nmae: 4.004, epoch: 59


In [212]:
eff0 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/eff0_baseline_check_data_driven_negative2.csv")
eff0

,img_name,leaf_weight
0,001.png,59.685313
1,002.png,290.030000
2,003.png,4.338477
3,004.png,70.264375
4,005.png,79.964375
...,...,...
455,456.png,62.201250
456,457.png,78.630625
457,458.png,29.054531
458,459.png,52.349688


In [221]:
base = pd.read_csv("D:/DACON_GROWTH/prediction_collection/effV2_eff0_negative_ensemble.csv")
base

,img_name,leaf_weight
0,001.png,56.260000
1,002.png,300.578750
2,003.png,4.577187
3,004.png,67.930312
4,005.png,76.690625
...,...,...
455,456.png,63.929063
456,457.png,77.781875
457,458.png,29.350078
458,459.png,48.681875


In [215]:
df0 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b0_ensemble.csv")
df1 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b1_ensemble.csv")
df2 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b2_ensemble.csv")
df3 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b3_ensemble.csv")
df4 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b4_ensemble.csv")
df5 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet-b5_ensemble.csv")
df6 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet-b6_ensemble.csv")
df_rw_m = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnetv2_rw_m_ensemble.csv")
df_rw_s = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnetv2_rw_s_ensemble.csv")
df_rw_t = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnetv2_rw_t_ensemble.csv")

In [216]:
samples1 = df_rw_s["leaf_weight"] * 0.97

In [217]:
samples2 = df6["leaf_weight"] * 0.97

In [53]:
#predict
from glob import glob
from collections import defaultdict
models={}
for i in glob("D:/DACON_GROWTH/models/*/*.ckpt"):
    models[i.split("\\")[1]] = i

for key, check in models.items():
    print(key)
    if key == "efficientnet_b0":
        temp1 = Regressor
        temp2 = 1280
        train_mode, test_mode = Augmentation_Policy(196, 224)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(224)
    if key == "efficientnet_b1":
        temp1 = Regressor
        temp2 = 1280
        train_mode, test_mode = Augmentation_Policy(224, 256)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(256)
    if key == "efficientnet_b2":
        temp1 = Regressor
        temp2 = 1408
        train_mode, test_mode = Augmentation_Policy(256, 288)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(288)
    if key == "efficientnet_b3":
        temp1 = Regressor
        temp2 = 1536
        train_mode, test_mode = Augmentation_Policy(288, 320)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(320)
    if key == "efficientnet_b4":
        temp1 = Regressor
        temp2 = 1792
        train_mode, test_mode = Augmentation_Policy(320, 384)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(384)
    if key == "efficientnetv2_rw_t":
        temp1 = Regressor
        temp2 = 1024
        train_mode, test_mode = Augmentation_Policy(224, 288)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(288)
    if key == "efficientnetv2_rw_s":
        temp1 = Regressor
        temp2 = 1792
        train_mode, test_mode = Augmentation_Policy(288, 384)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(384)
    if key == "efficientnetv2_rw_m":
        temp1 = Regressor
        temp2 = 2152
        train_mode, test_mode = Augmentation_Policy(320, 416)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(416)
    if key == "efficientnet-b5":
        temp1 = Regressor
        temp2 = 2048
        train_mode, test_mode = Augmentation_Policy(384, 456)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(456)
    if key == "efficientnet-b6":
        temp1 = Regressor
        temp2 = 2304
        train_mode, test_mode = Augmentation_Policy(456, 528)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(528)
    if key == "efficientnet-b7":
        temp1 = Regressor
        temp2 = 2560
        train_mode, test_mode = Augmentation_Policy(528, 600)
        train_path, train_imgs, test_path, test_imgs, augment_path, augment_imgs = Image_Data(600)

    test_dataset = PYTORCH_PREDICTSET(path=test_path, imgs=test_imgs, img2data=img2test, \
                                            transform=test_mode)
    model = LIGHTNING_MODEL.load_from_checkpoint(checkpoint_path = check, 
            model = temp1, model_type = key, out_features=temp2, 
            train_dataset = None, val_dataset = None, augment_dataset = None, 
            predict_dataset = test_dataset, train_batch = 16 , valid_batch = 16, 
            augment_batch = 16, predict_batch = 16, learning_rate=0)
    trainer = pl.Trainer(devices=[0], accelerator='gpu', max_epochs=1, precision=16)
    prediction = trainer.predict(model)
    sub_list = []
    for p in prediction:
        p = p.view(-1)
        for sample in p:
            sample = sample.cpu().numpy()
            sub_list.append(sample)
    sub = np.array(sub_list)
    sample_submission = pd.read_csv("D:/DACON_GROWTH/sample_submission.csv")
    sample_submission["leaf_weight"] = sub
    sample_submission.to_csv(f"D:/DACON_GROWTH/prediction_collection/{key}_nmae.csv", index=False)

efficientnet-b5
Loaded pretrained weights for efficientnet-b5


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\meta2\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:245: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

efficientnetv2_rw_s


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [220]:
sample = (samples1 + samples2) / 2
sample

0       53.81560
1      294.88000
2        5.54355
3       69.67025
4       80.97560
         ...    
455     66.80875
456     75.15560
457     24.35670
458     41.90885
459    258.11700
Name: leaf_weight, Length: 460, dtype: float64

In [84]:
eff0 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/eff0_baseline_check_data_driven_negative2.csv")
effv2m = pd.read_csv("D:/DACON_GROWTH/prediction_collection/effV2_rw_m_negative97.csv")
                     

In [91]:
result = (eff0["leaf_weight"].values*0.4)+ (effv2m["leaf_weight"].values*0.4)+(df5["leaf_weight"].values * 0.84*0.2)
result

array([5.59380800e+01, 2.98003000e+02, 4.60675000e+00, 6.77002500e+01,
       7.66237000e+01, 7.41177500e+01, 1.43709550e+00, 6.95101000e+01,
       9.70624312e-01, 7.65405800e+01, 5.88795509e-01, 3.20038300e+01,
       1.04971355e+01, 3.03442325e+01, 1.55634525e+01, 7.05620500e+01,
       7.27472000e+01, 3.20818400e+01, 7.58196500e+01, 2.84226400e+02,
       3.75046500e+01, 5.83925063e+00, 2.01765850e+00, 1.63782475e+01,
       6.48457500e+01, 7.77009500e+01, 1.30853200e+02, 4.06333500e+01,
       7.64054500e+01, 7.98793500e+01, 7.29981700e+01, 1.75954463e+01,
       2.13761109e+00, 5.81970362e+00, 1.18895912e+01, 1.56927600e+02,
       3.43309406e+00, 7.26950500e+01, 3.23666969e+00, 3.45734600e+01,
       7.59223500e+01, 9.90878000e+01, 1.42540428e+00, 2.94863400e+01,
       7.03725000e+01, 2.67811400e+01, 7.00555000e+01, 3.41535400e+01,
       5.79413500e+01, 7.06003300e+01, 1.83417000e+02, 6.64551700e+01,
       7.56450800e+01, 5.71155450e+01, 8.00764200e+01, 3.32072100e+01,
      

In [70]:
def np_nmae(y_true, y_pred):
    result = np.mean(np.abs(y_true-y_pred))/ np.mean(np.abs(y_true))
    return result

In [78]:
np_nmae(base["leaf_weight"].values, df5["leaf_weight"].values * 0.85)

0.05494667956194198

In [58]:
base = pd.read_csv("D:/DACON_GROWTH/prediction_collection/effV2_eff0_negative_ensemble.csv")
base

,img_name,leaf_weight
0,001.png,56.260000
1,002.png,300.578750
2,003.png,4.577187
3,004.png,67.930312
4,005.png,76.690625
...,...,...
455,456.png,63.929063
456,457.png,77.781875
457,458.png,29.350078
458,459.png,48.681875


In [181]:
dfn = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet-b5_nmae.csv")

In [101]:
df0 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b0_check.csv")
df1 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b1_check.csv")
df2 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b2_check.csv")
df3 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b3_check.csv")
df4 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet_b4_check.csv")
df5 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet-b5_check.csv")
df6 = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnet-b6_check.csv")
df_rw_m = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnetv2_rw_m_check.csv")
df_rw_s = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnetv2_rw_s_check.csv")
df_rw_t = pd.read_csv("D:/DACON_GROWTH/prediction_collection/efficientnetv2_rw_t_check.csv")

In [185]:
np_nmae(base["leaf_weight"].values, dfn["leaf_weight"].values * 0.85)

0.05494667956194198

In [113]:
np_nmae(base["leaf_weight"].values, df0["leaf_weight"].values * 0.95)

0.129645392569782

In [121]:
np_nmae(base["leaf_weight"].values, df1["leaf_weight"].values * 0.86)

0.08487612949052632

In [148]:
np_nmae(base["leaf_weight"].values, df3["leaf_weight"].values * 0.98)

0.08287135375130039

In [158]:
np_nmae(base["leaf_weight"].values, df4["leaf_weight"].values * 0.91)

0.07060538709825293

In [169]:
np_nmae(base["leaf_weight"].values, df5["leaf_weight"].values * 0.8)

0.05995703986519874

In [186]:
r1 = df1["leaf_weight"].values * 0.86
r3 = df3["leaf_weight"].values * 0.98
r4 = df4["leaf_weight"].values * 0.91
r5 = df5["leaf_weight"].values * 0.8

In [187]:
result = (base["leaf_weight"].values * 0.75)+ ((r1+r3+r4+r5)/4) * 0.25
result

array([ 56.443425  , 303.5793125 ,   5.02477187,  67.37280938,
        76.05594375,  74.33902812,   1.43988578,  69.20461875,
         1.12895854,  75.98796875,   0.77799354,  31.97255313,
        11.27301328,  30.36234297,  15.48400172,  70.71117187,
        71.5976    ,  31.92921875,  75.74040937, 290.5046875 ,
        37.36955938,   5.59242121,   1.91724109,  14.96813203,
        64.49685313,  77.06507812, 130.6540625 ,  41.98497187,
        76.49085937,  79.08879687,  73.34583438,  16.96719336,
         1.98676853,   5.6088309 ,  11.4937768 , 156.763375  ,
         3.21106231,  71.93948437,   3.04121283,  34.6030125 ,
        75.90597812,  96.89075   ,   1.61803589,  30.06840625,
        70.53249375,  27.7565875 ,  69.883375  ,  33.88511875,
        58.02777813,  70.85692188, 181.04775   ,  65.66887813,
        75.4401875 ,  56.79636094,  79.18309375,  33.67578437,
        77.17299687,  25.07654922,  36.78645313,  71.9325625 ,
         2.32865005,  32.36180781,  77.11610937,  79.49

In [182]:
np.mean([df0["leaf_weight"], df1["leaf_weight"], df2["leaf_weight"], df3["leaf_weight"],
        df4["leaf_weight"], df5["leaf_weight"], df6["leaf_weight"], 
        df_rw_m["leaf_weight"], df_rw_s["leaf_weight"], df_rw_t["leaf_weight"]],axis=0) 

array([ 60.389  , 328.08   ,   7.2961 ,  73.293  ,  83.769  ,  81.577  ,
         2.95415,  76.783  ,   2.95394,  83.159  ,   2.58871,  36.276  ,
        15.0786 ,  33.481  ,  19.33   ,  79.449  ,  78.39   ,  34.964  ,
        83.778  , 307.74   ,  37.668  ,   6.0783 ,   3.13936,  21.202  ,
        68.452  ,  82.685  , 144.56   ,  45.536  ,  86.415  ,  86.775  ,
        83.01   ,  19.223  ,   3.1896 ,   7.2183 ,  12.5731 , 175.83   ,
         3.57014,  80.74   ,   4.4703 ,  36.834  ,  86.497  , 112.685  ,
         3.00105,  31.958  ,  77.782  ,  31.301  ,  74.46   ,  35.645  ,
        67.702  ,  75.115  , 198.42   ,  69.613  ,  84.466  ,  62.009  ,
        85.942  ,  42.149  ,  83.474  ,  27.012  ,  40.561  ,  78.872  ,
         3.28664,  37.154  ,  86.764  ,  87.806  ,  84.934  ,  43.449  ,
        77.073  ,  81.93   , 140.045  ,  16.8659 , 278.04   ,  87.427  ,
        80.506  ,   3.53963,  15.1639 , 119.42   ,   2.69656,  77.454  ,
        71.881  ,  72.655  ,  75.589  ,  84.55   , 

In [188]:
sample_submission = pd.read_csv("D:/DACON_GROWTH/sample_submission.csv")

In [190]:
base

,img_name,leaf_weight
0,001.png,56.260000
1,002.png,300.578750
2,003.png,4.577187
3,004.png,67.930312
4,005.png,76.690625
...,...,...
455,456.png,63.929063
456,457.png,77.781875
457,458.png,29.350078
458,459.png,48.681875


In [189]:
sample_submission["leaf_weight"] = result
sample_submission

,img_name,leaf_weight
0,001.png,56.443425
1,002.png,303.579313
2,003.png,5.024772
3,004.png,67.372809
4,005.png,76.055944
...,...,...
455,456.png,64.412284
456,457.png,77.535044
457,458.png,29.074271
458,459.png,49.105944


In [184]:
sample_submission["leaf_weight"] =  np.mean([df0["leaf_weight"], df1["leaf_weight"], df2["leaf_weight"], df3["leaf_weight"],
        df4["leaf_weight"], df5["leaf_weight"], df6["leaf_weight"], 
        df_rw_m["leaf_weight"], df_rw_s["leaf_weight"], df_rw_t["leaf_weight"]],axis=0) 

In [94]:
sample_submission.to_csv(f"D:/DACON_GROWTH/prediction_collection/eff0effv2meff5_rate.csv", index=False)

In [98]:
sample_submission["leaf_weight"] = sample_submission["leaf_weight"] * 0.96

In [191]:
sample_submission.to_csv(f"D:/DACON_GROWTH/prediction_collection/final.csv", index=False)

In [100]:
sample_submission

,img_name,leaf_weight
0,001.png,53.700557
1,002.png,286.082880
2,003.png,4.422480
3,004.png,64.992240
4,005.png,73.558752
...,...,...
455,456.png,62.645040
456,457.png,74.912928
457,458.png,27.338940
458,459.png,46.343558
